<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/BERT/BERT_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Git Clone

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/BERT/data_in

/content/drive/MyDrive/Pytorch NLP/BERT/data_in


In [ ]:
ls

nsmc/  test_df.csv  train_df.csv


In [ ]:
!git clone 'https://github.com/kakaobrain/KorNLUDatasets.git'

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 16 (delta 1), reused 1 (delta 1), pack-reused 14
Unpacking objects: 100% (16/16), 46.67 MiB | 5.05 MiB/s, done.
Updating files: 100% (9/9), done.


In [ ]:
ls

KorNLUDatasets/  nsmc/


## Huggingface Transformers 라이브러리

In [ ]:
cd ..

/content/drive/MyDrive/Pytorch NLP/BERT


In [ ]:
ls

BERT_NLI.ipynb                  data_in/   transformers/
BERT_Text_Classification.ipynb  data_out/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00


# 데이터 분석

In [ ]:
import numpy as np
import pandas as pd
from transformers import *

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

## BertTokenizer 기반 문장 길이 분석

### 데이터 마련

In [20]:
multinli_df= pd.read_csv('data_in/KorNLUDatasets/KorNLI/multinli.train.ko.tsv',sep='\t',error_bad_lines=False)
multinli_df.head()

<ipython-input-20-5126d32c1d53>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  multinli_df= pd.read_csv('data_in/KorNLUDatasets/KorNLI/multinli.train.ko.tsv',sep='\t',error_bad_lines=False)
Skipping line 24426: expected 3 fields, saw 4
Skipping line 156343: expected 3 fields, saw 4
Skipping line 218766: expected 3 fields, saw 4
Skipping line 232318: expected 3 fields, saw 4
Skipping line 253493: expected 3 fields, saw 4

Skipping line 265734: expected 3 fields, saw 4
Skipping line 282588: expected 3 fields, saw 4
Skipping line 350969: expected 3 fields, saw 4



,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [19]:
len(multinli_df)

385494

In [24]:
snli_df=pd.read_csv('data_in/KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv',sep='\t',error_bad_lines=False)
snli_df.head()

<ipython-input-24-b96cd5b7fc92>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  snli_df=pd.read_csv('data_in/KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv',sep='\t',error_bad_lines=False)


,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [26]:
len(snli_df)

550152

In [25]:
train_df=pd.concat([multinli_df,snli_df],axis=0)
train_df.head()

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [28]:
len(train_df)

935646

In [30]:
sentences=pd.Series(train_df['sentence1'].tolist()+train_df['sentence2'].tolist())
sentences.head()

0           개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.
1    시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...
2                    우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.
3                         어떻게 아세요? 이 모든 것이 다시 그들의 정보다.
4    그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...
dtype: object

In [31]:
len(sentences)

1871292

### BertTokenizer 불러오기

In [ ]:
tokenizer=BertTokenizer('')